# Modelo 1: EfficientNetV2B3

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

## Carga de datos

1. Denifinir los directorios

In [2]:
train_dir = "/Users/inma/Desktop/tfg/datos/train"
test_dir = "/Users/inma/Desktop/tfg/datos/test"
val_dir = "/Users/inma/Desktop/tfg/datos/valid"

2. Establecer los generadores de imágenes

In [3]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255) # rescale = reducir la dimension
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

3. Extraer las imágenes de los directorios

In [4]:
train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224), # nueva escalacion
                                           class_mode = "categorical", # la clase es categorica: melanoma
                                           batch_size = 64 # tamaño del lote 
                                          ) 
test_data = test_gen.flow_from_directory(test_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 64
                                          )
val_data = val_gen.flow_from_directory(val_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 64
                                          )

Found 3219 images belonging to 3 classes.
Found 1072 images belonging to 3 classes.
Found 1072 images belonging to 3 classes.


## Modelo

1. Cargar el EfficientNetV2B3

In [18]:
# building a resnet model
model_base = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top= False) # hay muchos modelos

model_base.trainable = False # para que no entrene lo ya pre-entrenado

2. Crear el modelo

In [19]:
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3)) # puedo cambiar el tamaño???

x = model_base(inputs)
x = layers.GlobalAveragePooling2D()(x) # capa de pooling
x = layers.Dense(128, activation = "relu")(x) # capa muy conectada, con argumento de activacion
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(3, activation="softmax")(x)

model = tf.keras.Model(inputs,outputs) # modelo

3. Compilar el modelo

In [20]:
model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
    metrics = ["accuracy"]
)

4. Entrenamiento y validación

In [21]:
checkpointer = ModelCheckpoint(filepath = 'modelos/02_inceptionResNet_model_final.hdf5', 
                               verbose = 1, # monitoreo
                               save_best_only = True) # guarda solo el mejor

scheduler = ReduceLROnPlateau(monitor = 'val_loss', 
                              #min_delta = 0.005, # cuanto tiene que mejorar
                              factor = 0.1, # cuanto se reduce l_r*factor
                              patience = 3, # numero de epocas sin mejora para reducir l_r
                              min_lr = 1e-6, # l_r minimo
                              verbose = 1) # mostrar mensajes
 
early_stopper = EarlyStopping(monitor = 'val_loss', 
                              #min_delta = 0.005, # cuanto tiene que mejorar
                              patience = 10, # numero de epocas sin mejora para parar el train
                              verbose = 1, 
                              restore_best_weights = True) # establecer los pesos de la mejor epoca

In [22]:
model.fit(train_data,
                 epochs = 30,
                 steps_per_epoch = len(train_data),
                 validation_data = val_data,
                 validation_steps = len(val_data),
                 callbacks = [scheduler, early_stopper]
                 )

Epoch 1/30
51/51 [==============================] - 486s 9s/step - loss: 1.8067 - accuracy: 0.4088 - val_loss: 0.9743 - val_accuracy: 0.5159 - lr: 0.0100
Epoch 2/30
51/51 [==============================] - 490s 10s/step - loss: 0.9678 - accuracy: 0.5259 - val_loss: 0.9234 - val_accuracy: 0.5541 - lr: 0.0100
Epoch 3/30
 2/51 [>.............................] - ETA: 7:16 - loss: 0.8886 - accuracy: 0.6094

KeyboardInterrupt: 

In [44]:
model_base.trainable = True

for layer in model_base.layers[:-5]:
  model_base.trainable = False

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.01),
    metrics = ["accuracy"]
)

5. Entrenamiento sobre el modelo tuneado

In [45]:
scheduler_tuned = ReduceLROnPlateau(monitor = 'val_loss', 
                                    #min_delta = 0.005, # cuanto tiene que mejorar
                                    factor = 0.1, # cuanto se reduce l_r*factor
                                    patience = 3, # numero de epocas sin mejora para reducir l_r
                                    min_lr = 1e-6, # l_r minimo
                                    verbose = 1) # mostrar mensajes
 
early_stopper_tuned = EarlyStopping(monitor = 'val_loss', 
                              #min_delta = 0.005, # cuanto tiene que mejorar
                              patience = 10, # numero de epocas sin mejora para parar el train
                              verbose = 1, 
                              restore_best_weights = True) # establecer los pesos de la mejor epoca

In [46]:
model.fit(train_data,
                 epochs = 30,
                 steps_per_epoch = len(train_data),
                 validation_data = val_data,
                 validation_steps = len(val_data),
                 callbacks = [scheduler_tuned, early_stopper_tuned]
                 )

Epoch 1/30
51/51 [==============================] - 461s 9s/step - loss: 0.8820 - accuracy: 0.5707 - val_loss: 0.9197 - val_accuracy: 0.5746 - lr: 0.0100
Epoch 2/30
51/51 [==============================] - 481s 9s/step - loss: 0.8913 - accuracy: 0.5415 - val_loss: 0.8886 - val_accuracy: 0.5896 - lr: 0.0100
Epoch 3/30
51/51 [==============================] - 494s 10s/step - loss: 0.8841 - accuracy: 0.5601 - val_loss: 0.9038 - val_accuracy: 0.5504 - lr: 0.0100
Epoch 4/30
51/51 [==============================] - 501s 10s/step - loss: 0.8831 - accuracy: 0.5623 - val_loss: 0.8859 - val_accuracy: 0.5961 - lr: 0.0100
Epoch 5/30
51/51 [==============================] - 507s 10s/step - loss: 0.8674 - accuracy: 0.5676 - val_loss: 0.8471 - val_accuracy: 0.6185 - lr: 0.0100
Epoch 6/30
51/51 [==============================] - 515s 10s/step - loss: 0.8511 - accuracy: 0.5750 - val_loss: 0.8949 - val_accuracy: 0.6129 - lr: 0.0100
Epoch 7/30
51/51 [==============================] - 519s 10s/step - loss

## Guardar el modelo

In [17]:
model.save(
    'modelos/02_efficientNet_model_final',
    overwrite=True,
)

INFO:tensorflow:Assets written to: modelos/02_efficientNet_model_final/assets
